# Simple XRPL Wallet Money Functions

Two simple functions:
1. **Add money** to a wallet from testnet faucet
2. **Take money** from a wallet and send it away


In [1]:
# Setup
import os
import requests
from xrpl.clients import JsonRpcClient
from xrpl.wallet import Wallet
from xrpl.models.transactions import Payment
from xrpl.transaction import submit_and_wait
from xrpl.models.requests import AccountInfo

# Connect to testnet
client = JsonRpcClient("https://s.altnet.rippletest.net:51234")
print("✅ Connected to XRPL testnet")


✅ Connected to XRPL testnet


In [2]:
def add_money_to_wallet(public_key: str, private_key: str) -> bool:
    """
    Add 1000 XRP to a wallet from the testnet faucet.
    
    Args:
        public_key: The wallet's public key  
        private_key: The wallet's private key (seed)
        
    Returns:
        True if successful, False if failed
    """
    try:
        # Create wallet from keys to get address
        wallet = Wallet.from_seed(seed=private_key)
        wallet_address = wallet.classic_address
        
        print(f"🔍 Using public key: {public_key}")
        print(f"🔍 Derived address: {wallet_address}")
        
        # Try the main faucet first
        response = requests.post(
            "https://faucet.altnet.rippletest.net/accounts",
            json={"destination": wallet_address, "xrpAmount": 1000},
            headers={"Content-Type": "application/json"},
            timeout=30
        )
        
        if response.status_code == 200:
            print(f"✅ Added 1000 XRP to {wallet_address}")
            return True
        else:
            print(f"⚠️ Main faucet failed ({response.status_code}), trying alternative...")
            
            # Try alternative faucet method
            response2 = requests.post(
                "https://faucet.altnet.rippletest.net/accounts",
                data=f"destination={wallet_address}&amount=1000",
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                timeout=30
            )
            
            if response2.status_code == 200:
                print(f"✅ Added 1000 XRP to {wallet_address} (alternative method)")
                return True
            else:
                print(f"❌ Both faucet methods failed. Status codes: {response.status_code}, {response2.status_code}")
                print("💡 Try using the web faucet: https://faucet.altnet.rippletest.net/")
                return False
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        print("💡 Try using the web faucet: https://faucet.altnet.rippletest.net/")
        return False

# Test the function
print("Function ready! Use: add_money_to_wallet('public_key', 'private_key')")


Function ready! Use: add_money_to_wallet('public_key', 'private_key')


In [3]:
def take_money_from_wallet(public_key: str, private_key: str, amount_xrp: float, destination_address: str) -> bool:
    """
    Take money from a wallet and send it to another address.
    
    Args:
        public_key: The wallet's public key
        private_key: The wallet's private key (seed) 
        amount_xrp: Amount of XRP to take
        destination_address: Where to send the money
        
    Returns:
        True if successful, False if failed
    """
    try:
        # Create wallet from private key
        wallet = Wallet.from_seed(seed=private_key)
        
        print(f"🔍 Using public key: {public_key}")
        print(f"🔍 Wallet address: {wallet.classic_address}")
        
        # Convert XRP to drops (1 XRP = 1,000,000 drops)
        amount_drops = int(amount_xrp * 1_000_000)
        
        # Create payment transaction
        payment = Payment(
            account=wallet.classic_address,
            destination=destination_address,
            amount=str(amount_drops)
        )
        
        # Send the transaction
        response = submit_and_wait(payment, client, wallet)
        
        if response.is_successful():
            tx_hash = response.result.get("hash")
            print(f"✅ Took {amount_xrp} XRP from wallet. Transaction: {tx_hash}")
            return True
        else:
            print(f"❌ Failed to take money: {response.result}")
            return False
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

# Test the function
print("Function ready! Use: take_money_from_wallet('public_key', 'private_key', 10.0, 'destination_address')")


Function ready! Use: take_money_from_wallet('public_key', 'private_key', 10.0, 'destination_address')


In [4]:
def check_wallet_balance(wallet_address: str) -> float:
    """
    Check how much XRP a wallet has.
    
    Args:
        wallet_address: The wallet address to check
        
    Returns:
        Balance in XRP (0 if unfunded or error)
    """
    try:
        req = AccountInfo(account=wallet_address, ledger_index="validated")
        resp = client.request(req)
        
        if resp.is_successful():
            balance_drops = int(resp.result["account_data"]["Balance"])
            balance_xrp = balance_drops / 1_000_000  # Convert drops to XRP
            print(f"💰 Wallet {wallet_address} has {balance_xrp} XRP")
            return balance_xrp
        else:
            print(f"💰 Wallet {wallet_address} has 0 XRP (unfunded)")
            return 0.0
            
    except Exception as e:
        print(f"💰 Wallet {wallet_address} has 0 XRP (unfunded or error)")
        return 0.0

print("Function ready! Use: check_wallet_balance('your_wallet_address')")


Function ready! Use: check_wallet_balance('your_wallet_address')


## Example Usage


In [8]:
# COMPLETE EXAMPLE USAGE - START TO FINISH
print("🚀 COMPLETE EXAMPLE: Testing both functions from scratch")
print("=" * 60)

# STEP 1: Create a test wallet
print("🔨 STEP 1: Creating a fresh test wallet...")
new_wallet = Wallet.create()

# Extract the keys
private_key = new_wallet.seed
public_key = new_wallet.public_key
address = new_wallet.classic_address

print(f"✅ Created new wallet:")
print(f"   Private Key: {private_key}")
print(f"   Public Key:  {public_key}")
print(f"   Address:     {address}")

# STEP 2: Add money to the wallet
print(f"\n💰 STEP 2: Adding money to wallet...")
print(f"Calling: add_money_to_wallet(public_key, private_key)")

success = add_money_to_wallet(public_key, private_key)

if success:
    print(f"\n⏳ Waiting 5 seconds for funding to process...")
    import time
    time.sleep(5)
    
    # STEP 3: Check the balance
    print(f"\n💰 STEP 3: Checking wallet balance...")
    balance = check_wallet_balance(address)
    
    if balance > 10:
        # STEP 4: Take money from the wallet
        print(f"\n💸 STEP 4: Taking money from wallet...")
        destination = "rrrrrrrrrrrrrrrrrrrrrhoLvTp"  # Burn address (destroys XRP)
        amount = 5.0
        
        print(f"Calling: take_money_from_wallet(public_key, private_key, {amount}, burn_address)")
        take_success = take_money_from_wallet(public_key, private_key, amount, destination)
        
        if take_success:
            print(f"\n📊 STEP 5: Final balance check...")
            check_wallet_balance(address)
            
            print(f"\n🎉 SUCCESS! Both functions work!")
            print(f"   ✅ add_money_to_wallet() - Added money from faucet")
            print(f"   ✅ take_money_from_wallet() - Sent {amount} XRP to burn address")
        else:
            print(f"\n❌ Taking money failed")
    else:
        print(f"\n⚠️ Not enough balance to test taking money")
        
else:
    print(f"\n❌ Funding failed. Try manual funding:")
    print(f"   1. Go to: https://faucet.altnet.rippletest.net/")
    print(f"   2. Enter address: {address}")
    print(f"   3. Get 1000 XRP")
    print(f"   4. Then test taking money")
    
print(f"\n📋 YOUR WALLET DETAILS (save these!):")
print(f"   Private Key: {private_key}")
print(f"   Public Key:  {public_key}")
print(f"   Address:     {address}")

print(f"\n🔧 COPY-PASTE COMMANDS:")
print(f"add_money_to_wallet('{public_key}', '{private_key}')")
print(f"take_money_from_wallet('{public_key}', '{private_key}', 10.0, 'destination_address')")


🚀 COMPLETE EXAMPLE: Testing both functions from scratch
🔨 STEP 1: Creating a fresh test wallet...
✅ Created new wallet:
   Private Key: sEdVhu5ey3VHcJDtGBwu8XLLTejSXfG
   Public Key:  ED6872144D5FC67FBF3C533639B510AF02EFA4ED19D3763856D4FFE7812801FC62
   Address:     rUBAyhe6SDwg49ocFH34Coyk8wT1XuEE19

💰 STEP 2: Adding money to wallet...
Calling: add_money_to_wallet(public_key, private_key)
🔍 Using public key: ED6872144D5FC67FBF3C533639B510AF02EFA4ED19D3763856D4FFE7812801FC62
🔍 Derived address: rUBAyhe6SDwg49ocFH34Coyk8wT1XuEE19
⚠️ Main faucet failed (502), trying alternative...
❌ Both faucet methods failed. Status codes: 502, 502
💡 Try using the web faucet: https://faucet.altnet.rippletest.net/

❌ Funding failed. Try manual funding:
   1. Go to: https://faucet.altnet.rippletest.net/
   2. Enter address: rUBAyhe6SDwg49ocFH34Coyk8wT1XuEE19
   3. Get 1000 XRP
   4. Then test taking money

📋 YOUR WALLET DETAILS (save these!):
   Private Key: sEdVhu5ey3VHcJDtGBwu8XLLTejSXfG
   Public Key:  